# Assignment 9: GBDT

#### Response Coding: Example

<img src='http://i.imgur.com/TufZptV.jpg' width=700px>

> The response tabel is built only on train dataset.
> For a category which is not there in train data and present in test data, we will encode them with default values
Ex: in our test data if have State: D then we encode it as [0.5, 0.05]

<ol>
    <li><strong>Apply GBDT on these feature sets</strong>
        <ul>
            <li><font color='red'>Set 1</font>: categorical(instead of one hot encoding, try <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/handling-categorical-and-numerical-features/'>response coding</a>: use probability values), numerical features + project_title(TFIDF)+  preprocessed_eassay (TFIDF)+sentiment Score of eassay(check the bellow example, include all 4 values as 4 features)</li>
            <li><font color='red'>Set 2</font>: categorical(instead of one hot encoding, try <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/handling-categorical-and-numerical-features/'>response coding</a>: use probability values), numerical features + project_title(TFIDF W2V)+  preprocessed_eassay (TFIDF W2V)</li>        </ul>
    </li>
    <li><strong>The hyper paramter tuning (Consider any two hyper parameters)</strong>
        <ul>
    <li>Find the best hyper parameter which will give the maximum <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/receiver-operating-characteristic-curve-roc-curve-and-auc-1/'>AUC</a> value</li>
    <li>find the best hyper paramter using k-fold cross validation/simple cross validation data</li>
    <li>use gridsearch cv or randomsearch cv or you can write your own for loops to do this task</li>
        </ul>
    </li>
    <li>
    <strong>Representation of results</strong>
        <ul>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/Gp2DQmh.jpg' width=500px> with X-axis as <strong>n_estimators</strong>, Y-axis as <strong>max_depth</strong>, and Z-axis as <strong>AUC Score</strong> , we have given the notebook which explains how to plot this 3d plot, you can find it in the same drive <i>3d_scatter_plot.ipynb</i></li>
            <p style="text-align:center;font-size:30px;color:red;"><strong>or</strong></p> <br>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/fgN9aUP.jpg' width=300px> <a href='https://seaborn.pydata.org/generated/seaborn.heatmap.html'>seaborn heat maps</a> with rows as <strong>n_estimators</strong>, columns as <strong>max_depth</strong>, and values inside the cell representing <strong>AUC Score</strong> </li>
    <li>You choose either of the plotting techniques out of 3d plot or heat map</li>
    <li>Once after you found the best hyper parameter, you need to train your model with it, and find the AUC on test data and plot the ROC curve on both train and test.
    <img src='https://i.imgur.com/wMQDTFe.jpg' width=300px></li>
    <li>Along with plotting ROC curve, you need to print the <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/confusion-matrix-tpr-fpr-fnr-tnr-1/'>confusion matrix</a> with predicted and original labels of test data points
    <img src='https://i.imgur.com/IdN5Ctv.png' width=300px></li>
            </ul>
    <br>
    <li>You need to summarize the results at the end of the notebook, summarize it in the table format
        <img src='http://i.imgur.com/YVpIGGE.jpg' width=400px>
    </li>
</ol>

In [1]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# import nltk
# nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

for_sentiment = 'a person is a person no matter how small dr seuss i teach the smallest students with the biggest enthusiasm \
for learning my students learn in many different ways using all of our senses and multiple intelligences i use a wide range\
of techniques to help all my students succeed students in my class come from a variety of different backgrounds which makes\
for wonderful sharing of experiences and cultures including native americans our school is a caring community of successful \
learners which can be seen through collaborative student project based learning in and out of the classroom kindergarteners \
in my class love to work with hands on materials and have many different opportunities to practice a skill before it is\
mastered having the social skills to work cooperatively with friends is a crucial aspect of the kindergarten curriculum\
montana is the perfect place to learn about agriculture and nutrition my students love to role play in our pretend kitchen\
in the early childhood classroom i have had several kids ask me can we try cooking with real food i will take their idea \
and create common core cooking lessons where we learn important math and writing concepts while cooking delicious healthy \
food for snack time my students will have a grounded appreciation for the work that went into making the food and knowledge \
of where the ingredients came from as well as how it is healthy for their bodies this project would expand our learning of \
nutrition and agricultural cooking recipes by having us peel our own apples to make homemade applesauce make our own bread \
and mix up healthy plants from our classroom garden in the spring we will also create our own cookbooks to be printed and \
shared with families students will gain math and literature skills as well as a life long enjoyment for healthy cooking \
nannan'
ss = sid.polarity_scores(for_sentiment)

for k in ss:
    print('{0}: {1}, '.format(k, ss[k]), end='')

# we can use these 4 things as features/attributes (neg, neu, pos, compound)
# neg: 0.0, neu: 0.753, pos: 0.247, compound: 0.93

neg: 0.01, neu: 0.745, pos: 0.245, compound: 0.9975, 

<h1>1. GBDT (xgboost/lightgbm) </h1>

## 1.1 Loading Data

In [2]:
import pandas
data = pandas.read_csv('preprocessed_data.csv')

In [5]:
data.head()

,school_state,teacher_prefix,project_grade_category,teacher_number_of_previously_posted_projects,project_is_approved,clean_categories,clean_subcategories,essay,price
0,ca,mrs,grades_prek_2,53,1,math_science,appliedsciences health_lifescience,i fortunate enough use fairy tale stem kits cl...,725.05
1,ut,ms,grades_3_5,4,1,specialneeds,specialneeds,imagine 8 9 years old you third grade classroo...,213.03
2,ca,mrs,grades_prek_2,10,1,literacy_language,literacy,having class 24 students comes diverse learner...,329.00
3,ga,mrs,grades_prek_2,2,1,appliedlearning,earlydevelopment,i recently read article giving students choice...,481.04
4,wa,mrs,grades_3_5,2,1,literacy_language,literacy,my students crave challenge eat obstacles brea...,17.74


## Defining Reusable Functions

In [4]:
# Plotting Confusion Matrix
def plot_matrix(y_true, y_pred):
    '''Input: true labes , predicted labels
       Output: None
       Functionality: Plots the confusion, precison and recall matrices
    '''
    conf = confusion_matrix(y_true, y_pred)
    # Column Sum = 1
    precision = conf/conf.sum(0)
    # Row Sum = 1
    recall    = (conf.T/conf.sum(1)).T
    cmap='YlGnBu'
    labels = [1,2,3,4,5,6,7,8,9]
    print('-'*20,'Confusion Matrix','-'*20)
    plt.figure(figsize=(20,7))
    sns.heatmap(conf ,annot=True, fmt='.3f', cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('Original')
    plt.show()
    print('-'*20,'Precision Matrix ( Columns Sum == 1 )','-'*20)
    plt.figure(figsize=(20,7))
    sns.heatmap(precision ,annot=True, fmt='.3f', cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('Original')
    plt.show()
    print('-'*20,'Recall Matrix ( Row Sum == 1 )','-'*20)
    plt.figure(figsize=(20,7))
    sns.heatmap(recall ,annot=True, fmt='.3f', cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('Original')
    plt.show()
    
# Defining Functions for Categorical Columns Response Encoding
def response_encoding_fitting(x_train,y_train,feature,label,alpha):
    '''
    Input   : x_train, y_train, feature, label, alpha
    Output  : dictionary containing response coded features
    Functionality: Encoding a feature using response encoding feature techniques
    '''
    temp = x_train.copy()
    temp[label] = y_train
    temp = temp.groupby([feature,label])[label].agg(Total='count').reset_index().sort_values([feature,label])
    response_encoding = {}
    # For evey Gene
    for gene in temp[feature].unique():
        response = np.zeros((9,1))
        # Filter DataFrame Values for that gene
        x = temp[temp[feature] == gene]
        total = x['Total'].sum()
        # For each class present for the gene 
        response_check = (x['Total'].values + 10 *alpha) / total + 90 * alpha

        for i in temp[label].unique():
            z=x[x[label] == i]['Total']
            if len(z.values) == 0:
                z=0
            numerator   = (z + 10 * alpha)
            denominator = (total +( 90 * alpha))
            response[i-1] = (numerator)/(denominator)
        response_encoding[gene] = response.T[0]
    return response_encoding
    
def response_encoding_fit_transform(x,y,feature,label,alpha):
    '''
    Input   : x, y, feature, label
    Output  : response encoded data
    Functionality: fit and Transforming the data into response encoded data
    '''
    
    temp = x.copy()
    temp[label] = y

    response_dictionary = response_encoding_fitting(x, y, feature, label, alpha)
    
    final_feature = []

    for ind, row in temp.iterrows():
        if row[feature] in response_dictionary.keys():
            final_feature.append(response_dictionary[row[feature]])
        else:
            feature_count = len(df[feature].unique())
            final_feature.append(np.ones(feature_count)/feature_count)
    return np.array(final_feature)

# Creating Response Encoding Functions for TEXT
def creating_word_count_dict(df):
    return dict(Counter(' '.join(df['Text'].tolist()).split()))

def response_encoding_text(curr_df,label):
    total_wc_dict = creating_word_count_dict(curr_df)
    response_encoding_feature = np.zeros((curr_df.shape[0],9))
    for cls in range(1,10):
        # print('calculating for class {}'.format(cls))
        temp_df = curr_df[label['Class'] == cls]
        # print(temp_df.shape)
        per_class_wc_dict = creating_word_count_dict(temp_df)
        index = 0
        for ind, row in curr_df.iterrows():
            sum_prob = 0
            for word in row['Text'].split():
                # print('Calculation for word = {}'.format(word))
                temp = ((per_class_wc_dict.get(word,0)+10)/(total_wc_dict.get(word,0) + 90) )
                # print(temp)
                sum_prob += math.log( temp )
            response_encoding_feature[index][cls-1] = math.exp(sum_prob/len(row['Text'].split()))
            index += 1
    return response_encoding_feature

def create_tfidf_w2v(df, col):
    
    with open('drive/My Drive/Colab Notebooks/AppliedAICourse/Assignment/glove_vectors', 'rb') as f:
        model = pickle.load(f)
        glove_words =  set(model.keys())

    # Creating TFIDF 
    tfidf_vec = TfidfVectorizer()
    tfidf_vec.fit(df[col])
    idf_dict = dict(zip(tfidf_vec.get_feature_names(), list(tfidf_vec.idf_)))
    tfidf_words = set(tfidf_vec.get_feature_names())

    # Creating TVIDF weigthed W2V
    tfidf_w2v_vectors = []
    for sentence in tqdm(df[col].values):
        vector = np.zeros(300)
        tfidf_val = 0
        for word in sentence.split():
            if word in glove_words and word in tfidf_words:
                temp = model[word]
                # Calcualting the tfidf values
                tf_idf = idf_dict[word] * (sentence.count(word)/len(sentence.split()))
                vector += temp * tf_idf
                tfidf_val += tf_idf
        if tfidf_val != 0:
            vector /= tfidf_val
        tfidf_w2v_vectors.append(vector)
    return tfidf_w2v_vectors



def transforming(x_train, y_train, x_test, y_test, x_cv, y_cv, col):
    # Transforming Integer Fields
    if x_train[col].dtype == np.dtype('int64') or x_train[col].dtype == np.dtype('float64'):
        
        std = StandardScaler()
        x_train_val = std.fit_transform(x_train[col].values.reshape(-1,1))
        x_cv_val    = std.transform(x_cv[col].values.reshape(-1,1))
        x_test_val  = std.transform(x_test[col].values.reshape(-1,1))
        
   # Transfroming 
    if x_train[col].dtype == np.dtype('object'):
        
        if col == 'essay':
            tf_vec      = TfidfVectorizer(ngram_range=(1,5),min_df=10,norm='l2')
            x_train_val = tf_vec.fit_transform(x_train[col].values).tocsr()
            x_test_val    = tf_vec.transform(x_test[col].values).tocsr()
            x_cv_val  = tf_vec.transform(x_cv[col].values).tocsr()
                
        else:
            response_encoding_fit_transform(x_train, y_train, col, '')
        
    return (x_train_val, x_cv_val, x_test_val)

<h2>1.2 Splitting data into Train and cross validation(or test): Stratified Sampling</h2>

In [0]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h2>1.3 Make Data Model Ready: encoding eassay, and project_title</h2>

In [0]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h2>1.4 Make Data Model Ready: encoding numerical, categorical features</h2>

In [0]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding 
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h2>1.5 Appling Models on different kind of featurization as mentioned in the instructions</h2>

<br>Apply GBDT on different kind of featurization as mentioned in the instructions
<br> For Every model that you work on make sure you do the step 2 and step 3 of instrucations

In [0]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h1>3. Summary</h1>

<br> as mentioned in the step 4 of instructions